Note: to run this script, the NSIDC library has been adapted. If you want to download AQUA-AMSRE images, please sure to change the unchecked lines of the def make_measures_url(url_template, res, dt, freq, HV, AD): function. The suffix is different for the DMSP vs. AQUA situation, hence this temporary solution

In [1]:
%reload_ext autoreload
%matplotlib inline
%load_ext autoreload
%autoreload 2

import os
import cartopy
import cartopy.crs as ccrs
import cartopy.feature as cfeature
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER

import matplotlib.pyplot as plt
import xarray as xr
import pyproj
import osr
import datetime
import subprocess
import shutil
import numpy as np

# import our own small lib
import nsidc_dmsp #adjusted from original in suffix (can also be included in the if/else statements)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [6]:
url_base = 'https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/'
url_folder = '{:s}/'
url_template = url_base + url_folder + 'NSIDC-0630-EASE2_T{:s}km-F08_SSMI-{:s}-{:s}{:s}-{:s}-{:s}-v1.3.nc'
# Specify between S(outhern) or N(orthern). T(ropical) is for AMSR-E
#A vs. D AQUA is E vs. M. SSMI
download_template = 'wget --http-user={:s} --http-password={:s} --load-cookies mycookies.txt --save-cookies mycookies.txt --keep-session-cookies --no-check-certificate --auth-no-challenge -r --reject "index.html*" -np -e robots=off {:s}'

fn_out_template = 'NSIDC-0630-EASE2_T{:s}km-F08_SSMI-{:s}{:s}-{:s}_{:04d}.nc'
out_path = os.path.abspath('netcdf_dmsp')
if not(os.path.isdir(out_path)):
    # prepare the output path
    os.makedirs(out_path)
freq = '37' # 36 GhZ following Brakenridge 2007 (WRR M/C ratio paper)
res = '25'  # can also be 12.5 or 25, although this depends on the chosen frequency as well
HV = 'H' # Horizontal or Vertical polarisation
AD = 'D' # MORINING OR EVENING Ascending or descending, descending contains imagery during the day, probably showing more contrast

# start_date = datetime.datetime(2002, 6, 1)
start_date = datetime.datetime(1987, 1, 1)
end_date = datetime.datetime(1992, 1, 1)
# start_date = datetime.datetime(2010, 1, 1) # date after data recovery
#end_date = datetime.datetime(2002, 6, 5)  # only a few days to make a demonstration
# end_date = datetime.datetime(2011, 12, 31)  # use this end date to cover the whole AMSR-E period.

step = datetime.timedelta(days=1)  # M

try:
    credential_fn = os.path.abspath('../../nsidc_credentials.txt')
    username, password = str(np.loadtxt(credential_fn, dtype=np.str)).split(',')
except:
    print('No credential file found, please put a txt file with <user>,<pass> in {:s}'.format(credential_fn))
 

In [7]:
print(' Frequency:',freq,'GHz.', 
      ' Resolution:',res, 'km.',
      ' Polarization:', HV,'.' 
      ' Ascending/descending:', AD)

 Frequency: 37 GHz.  Resolution: 25 km.  Polarization: H . Ascending/descending: D


In [8]:
# define the proj4string belonging to the grids you are downloading. You can find these in a downloaded sample 
# under the variable/attribute crs.proj4text. Here we simply copy-paste that.
proj4str = '+proj=cea +lat_0=0 +lon_0=0 +lat_ts=30 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m'
points_interest = [(2.4406387000000223, 6.893251099999999),
                   (2.2999999999999545, 7.099999999999999),
                   (2.4851462999999967, 8.0335995),
                   (2.0499999999999545, 7.566667)
]

# In this example, we are interested in parts of West-Africa, so we provide a bounding box covering that in the format
# [(xmin, ymin), (xmax, ymax)]
# bounds = [(-5., 0.),
#           (10., 15.),
#          ]
bounds = [(2., 6.),
          (3., 8.),
         ]

# we define a projection object for lat-lon WGS84 (EPSG code 4326)
proj_out = pyproj.Proj(init='epsg:4326')
# we define a projection object for the projection used in the downloaded grids.
proj_in = pyproj.Proj(proj4str)

# here we convert the coordinates in lat-lon into the coordinate system of the downloaded grids.
bounds_xy = nsidc_dmsp.proj_coords(bounds, proj_out, proj_in)
points_xy = nsidc_dmsp.proj_coords(points_interest, proj_out, proj_in)

# now we're all set!

In [9]:
import time
dt = start_date
list_ds = []
year = dt.year # let's store data per year

while dt <= end_date:
    url = nsidc_dmsp.make_measures_url(url_template, res, dt, freq, HV, AD)  # make the url string to download
    fn = url.strip('https://')  # strip https:// from the url to get the local location of the downloaded file
    path = fn.split('/')[0]  # split fn in little pieces on / and keep only the 0-th indexed piece (main folder)
    download = nsidc_dmsp.make_measures_download(download_template, url, username, password)  # pass the url to a download string template
    success = subprocess.call(download)  # call the download string in a command-line (use wget.exe! get it from online)
    if success == 0:
        print('Retrieved {:s}'.format(url))
        # the file was successfully downloaded (if not zero, there was a problem or file is sinmply not available)
        # read file, cut a piece and add it to our list of time steps
        ds = xr.open_dataset(fn, decode_cf=False)
        ds_sel = nsidc_dmsp.select_bounds(ds, bounds_xy)
        list_ds.append(ds_sel.load())  # load the actual data so that we can delete the original downloaded files
        ds.close()
        #time.sleep(0.001)
        shutil.rmtree(path)  # we're done cutting part of the required grid, so throw away the originally downloaded world grid.
    dt += step   # increase by one day to go for the next download day.
    if (year != dt.year) or (dt > end_date):  # store results if one moves to a new year or the end date is reached
        # concatenate the list of timesteps into a new ds
        if len(list_ds) > 0:
            # only store if any values were found
            ds_year = nsidc_dmsp.correct_miss_fill(xr.concat(list_ds, dim='time'))
            # ds_year.data_vars
            encoding = {var: {'zlib': True} for var in ds_year.data_vars if var != 'crs'}
            # store the current dataset into a nice netcdf file
            fn_out = os.path.abspath(os.path.join(out_path, fn_out_template.format(str(res), freq, HV, AD, year)))
            print('Writing output for year {:d} to {:s}'.format(year, fn_out))
            ds_year.to_netcdf(fn_out, encoding=encoding)
        # prepare a new dataset
        list_ds = []  # empty list
        year = dt.year  # update the year


Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.09/NSIDC-0630-EASE2_T25km-F08_SSMI-1987190-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.10/NSIDC-0630-EASE2_T25km-F08_SSMI-1987191-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.11/NSIDC-0630-EASE2_T25km-F08_SSMI-1987192-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.12/NSIDC-0630-EASE2_T25km-F08_SSMI-1987193-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.13/NSIDC-0630-EASE2_T25km-F08_SSMI-1987194-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.14/NSIDC-0630-EASE2_T25km-F08_SSMI-1987195-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.07.15/NSIDC-0630-EASE2_T25km-F08_SSMI-1987196-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.07/NSIDC-0630-EASE2_T25km-F08_SSMI-1987250-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.08/NSIDC-0630-EASE2_T25km-F08_SSMI-1987251-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.09/NSIDC-0630-EASE2_T25km-F08_SSMI-1987252-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.10/NSIDC-0630-EASE2_T25km-F08_SSMI-1987253-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.11/NSIDC-0630-EASE2_T25km-F08_SSMI-1987254-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.12/NSIDC-0630-EASE2_T25km-F08_SSMI-1987255-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.09.13/NSIDC-0630-EASE2_T25km-F08_SSMI-1987256-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.06/NSIDC-0630-EASE2_T25km-F08_SSMI-1987310-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.07/NSIDC-0630-EASE2_T25km-F08_SSMI-1987311-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.08/NSIDC-0630-EASE2_T25km-F08_SSMI-1987312-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.09/NSIDC-0630-EASE2_T25km-F08_SSMI-1987313-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.10/NSIDC-0630-EASE2_T25km-F08_SSMI-1987314-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.11/NSIDC-0630-EASE2_T25km-F08_SSMI-1987315-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1987.11.12/NSIDC-0630-EASE2_T25km-F08_SSMI-1987316-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.04/NSIDC-0630-EASE2_T25km-F08_SSMI-1988004-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.05/NSIDC-0630-EASE2_T25km-F08_SSMI-1988005-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.06/NSIDC-0630-EASE2_T25km-F08_SSMI-1988006-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.07/NSIDC-0630-EASE2_T25km-F08_SSMI-1988007-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.08/NSIDC-0630-EASE2_T25km-F08_SSMI-1988008-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.09/NSIDC-0630-EASE2_T25km-F08_SSMI-1988009-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.01.10/NSIDC-0630-EASE2_T25km-F08_SSMI-1988010-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.04/NSIDC-0630-EASE2_T25km-F08_SSMI-1988064-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.05/NSIDC-0630-EASE2_T25km-F08_SSMI-1988065-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.06/NSIDC-0630-EASE2_T25km-F08_SSMI-1988066-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.07/NSIDC-0630-EASE2_T25km-F08_SSMI-1988067-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.08/NSIDC-0630-EASE2_T25km-F08_SSMI-1988068-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.09/NSIDC-0630-EASE2_T25km-F08_SSMI-1988069-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1988.03.10/NSIDC-0630-EASE2_T25km-F08_SSMI-1988070-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

...

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.15/NSIDC-0630-EASE2_T25km-F08_SSMI-1991105-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.16/NSIDC-0630-EASE2_T25km-F08_SSMI-1991106-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.17/NSIDC-0630-EASE2_T25km-F08_SSMI-1991107-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.18/NSIDC-0630-EASE2_T25km-F08_SSMI-1991108-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.19/NSIDC-0630-EASE2_T25km-F08_SSMI-1991109-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.20/NSIDC-0630-EASE2_T25km-F08_SSMI-1991110-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.04.21/NSIDC-0630-EASE2_T25km-F08_SSMI-1991111-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.14/NSIDC-0630-EASE2_T25km-F08_SSMI-1991165-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.15/NSIDC-0630-EASE2_T25km-F08_SSMI-1991166-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.16/NSIDC-0630-EASE2_T25km-F08_SSMI-1991167-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.17/NSIDC-0630-EASE2_T25km-F08_SSMI-1991168-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.18/NSIDC-0630-EASE2_T25km-F08_SSMI-1991169-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.19/NSIDC-0630-EASE2_T25km-F08_SSMI-1991170-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.06.20/NSIDC-0630-EASE2_T25km-F08_SSMI-1991171-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.13/NSIDC-0630-EASE2_T25km-F08_SSMI-1991225-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.14/NSIDC-0630-EASE2_T25km-F08_SSMI-1991226-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.15/NSIDC-0630-EASE2_T25km-F08_SSMI-1991227-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.16/NSIDC-0630-EASE2_T25km-F08_SSMI-1991228-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.17/NSIDC-0630-EASE2_T25km-F08_SSMI-1991229-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.18/NSIDC-0630-EASE2_T25km-F08_SSMI-1991230-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.08.19/NSIDC-0630-EASE2_T25km-F08_SSMI-1991231-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.12/NSIDC-0630-EASE2_T25km-F08_SSMI-1991285-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.13/NSIDC-0630-EASE2_T25km-F08_SSMI-1991286-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.14/NSIDC-0630-EASE2_T25km-F08_SSMI-1991287-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.15/NSIDC-0630-EASE2_T25km-F08_SSMI-1991288-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.16/NSIDC-0630-EASE2_T25km-F08_SSMI-1991289-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.17/NSIDC-0630-EASE2_T25km-F08_SSMI-1991290-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.10.18/NSIDC-0630-EASE2_T25km-F08_SSMI-1991291-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.11/NSIDC-0630-EASE2_T25km-F08_SSMI-1991345-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.12/NSIDC-0630-EASE2_T25km-F08_SSMI-1991346-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.13/NSIDC-0630-EASE2_T25km-F08_SSMI-1991347-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.14/NSIDC-0630-EASE2_T25km-F08_SSMI-1991348-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.15/NSIDC-0630-EASE2_T25km-F08_SSMI-1991349-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.16/NSIDC-0630-EASE2_T25km-F08_SSMI-1991350-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsidc.org/MEASURES/NSIDC-0630.001/1991.12.17/NSIDC-0630-EASE2_T25km-F08_SSMI-1991351-37H-D-GRD-CSU-v1.3.nc
Retrieved https://n5eil01u.ecs.nsi

In [25]:
path = fn.split('/')[1]
path

'MEASURES'